# 生成型LLMを用いた固有表現認識

## Swallowを用いた固有表現認識の実装

### 準備

In [ ]:
!pip install bitsandbytes datasets seqeval transformers[ja,torch] openai=="1.56.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 105.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

## データセット・前処理

### データセットの読み込み

In [ ]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

ner-wikipedia-dataset.py:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from pprint import pprint

# 検証セットの1つ目のデータを表示する
val_data = dataset["validation"][0]
pprint(val_data)

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


### データの前処理

In [ ]:
def convert_data_format(data: dict[str, str]) -> dict[str, str]:
    """データフォーマットを変換する"""
    data["input"] = data["text"]
    data["output"] = "\n".join(
        [
            f"{i+1}. | "
            + " | ".join([f"{v}" for k, v in entity.items() if k != "span"])
            for i, entity in enumerate(data["entities"])
        ]
    )
    return data

# データフォーマットを変換する
val_data = convert_data_format(val_data)
pprint(val_data)

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'input': '「復活篇」はグリーンバニーからの発売となっている。',
 'output': '1. | 復活篇 | 製品名\n2. | グリーンバニー | 法人名',
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


### プロンプトテンプレートの作成

In [ ]:
def create_prompt_template(
    instruction: str, few_shots: list[dict[str, str]] | None = None
) -> str:
    """プロンプトテンプレートを作成する"""
    prompt_template = (
        "以下は、タスクを説明する指示と、"
        "文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
    )
    prompt_template += f"### 指示:\n{instruction}\n\n"
    if few_shots is not None:
        for few_shot in few_shots:
            prompt_template += f"### 入力:\n{few_shot['input']}\n\n"
            prompt_template += f"### 応答:\n{few_shot['output']}\n\n"
    prompt_template += "### 入力:\n{input}\n\n"
    prompt_template += "### 応答:\n"
    return prompt_template

# 指示文を指定してプロンプトテンプレートを作成する
entity_types = sorted(
    set([e["type"] for entity in dataset["train"]["entities"] for e in entity])
)
instruction = (
    "テキストを入力とし、テキストの中から出現順に固有表現を抽出してください。"
    "回答は固有表現名、固有表現タイプを含めてください。"
    "固有表現タイプは{entity_types}から選択してください。".format(
        entity_types="・".join(entity_types)
    )
)

# 訓練セットをシャッフルする
train_dataset = dataset["train"].shuffle()
# 訓練セットの前処理をする
train_dataset = train_dataset.map(convert_data_format)
# 30件のfew-shot事例を取得する
few_shots = list(train_dataset)[:30]
# プロンプトテンプレートを作成する
prompt_template = create_prompt_template(instruction, few_shots)
print(prompt_template)

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
テキストを入力とし、テキストの中から出現順に固有表現を抽出してください。回答は固有表現名、固有表現タイプを含めてください。固有表現タイプはその他の組織名・イベント名・人名・地名・政治的組織名・施設名・法人名・製品名から選択してください。

### 入力:
この試合では先制適時打、本塁打を放つ。

### 応答:


### 入力:
「野菊の墓」は、伊藤左千夫の同名小説を原作とし、1981年8月8日に公開された東映東京撮影所、サンミュージック製作、東映配給の日本映画である。

### 応答:
1. | 野菊の墓 | 製品名
2. | 伊藤左千夫 | 人名
3. | 東映 | 法人名
4. | 東京撮影所 | 施設名
5. | サンミュージック | 法人名
6. | 東映 | 法人名
7. | 日本 | 地名

### 入力:
また、マヌエル・ノイアー、アリエン・ロッベンを抑えて2014年のUEFA欧州最優秀選手賞も受賞した。

### 応答:
1. | マヌエル・ノイアー | 人名
2. | アリエン・ロッベン | 人名
3. | UEFA欧州最優秀選手賞 | 製品名

### 入力:
彼は1981年に暗黒物質や銀河の回転曲線問題に対して修正ニュートン力学を提唱したことで著名である。

### 応答:


### 入力:
以後もクラブでは守備力の問題等を解決できず、チーム内での序列はマルセロより下で控えに甘んじ、2010年7月末にリヴァプールFCへの移籍願望を口にしたが公式決定には至らず、翌8月にエルクレスCFにレンタル移籍した。

### 応答:
1. | マルセロ | 人名
2. | リヴァプールFC | その他の組織名
3. | エルクレスCF | その他の組織名

### 入力:
その後2001年までに東京支店・郡山営業所・横浜営業所を開設、同年8月には、本州の大手ホームセンターであるジョイフル本田やアークランドサカモトと共にジョイフルエーケーを立ち上げ、ホームセンター事業を拡大した。

### 応答:
1. | 東京支店 | 施設名
2. | 郡山営業所 | 施設名
3. | 横浜営業所 | 施設名
4. | 本州 | 地名
5. | ジョ

### プロンプトテンプレートへ入力テキストの挿入

In [ ]:
def insert_text_to_prompt_template(
    data: dict[str, str], prompt_template: str
) -> dict[str, str]:
    """入力テキストをプロンプトテンプレートに挿入する"""
    data["prompt"] = prompt_template.format(input=data["input"])
    return data

# 検証セットのデータで入力テキストをプロンプトテンプレートに挿入する
val_data = insert_text_to_prompt_template(val_data, prompt_template)
pprint(val_data)

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'input': '「復活篇」はグリーンバニーからの発売となっている。',
 'output': '1. | 復活篇 | 製品名\n2. | グリーンバニー | 法人名',
 'prompt': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n'
           '\n'
           '### 指示:\n'
           'テキストを入力とし、テキストの中から出現順に固有表現を抽出してください。回答は固有表現名、固有表現タイプを含めてください。固有表現タイプはその他の組織名・イベント名・人名・地名・政治的組織名・施設名・法人名・製品名から選択してください。\n'
           '\n'
           '### 入力:\n'
           'この試合では先制適時打、本塁打を放つ。\n'
           '\n'
           '### 応答:\n'
           '\n'
           '\n'
           '### 入力:\n'
           '「野菊の墓」は、伊藤左千夫の同名小説を原作とし、1981年8月8日に公開された東映東京撮影所、サンミュージック製作、東映配給の日本映画である。\n'
           '\n'
           '### 応答:\n'
           '1. | 野菊の墓 | 製品名\n'
           '2. | 伊藤左千夫 | 人名\n'
           '3. | 東映 | 法人名\n'
           '4. | 東京撮影所 | 施設名\n'
           '5. | サンミュージック | 法人名\n'
           '6. | 東映 | 法人名\n'
           '7. | 日本 | 地名\n'
           '\

## 固有表現の予測・抽出

### テキスト生成パイプラインの作成

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

model_name = "tokyotech-llm/Swallow-7b-instruct-hf"
# AutoTokenizerでトークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)
# モデルを量子化して読み込むためのパラメータを指定する
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# 生成を行うモデルであるAutoModelForCausalLMを使ってモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)
# テキスト生成用のパラメータを指定する
generation_config = {
    "max_new_tokens": 512, # 生成する最大トークン数
    "top_p": 1.0, # top-pサンプリング
    "repetition_penalty": 1.0, # 繰り返しペナルティ
}
# pipelineを作成する
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config
)

tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

### 固有表現認識の実行

In [ ]:
# 固有表現認識を行う
output = text_generation_pipeline(val_data["prompt"])
# プロンプト部分を削除して予測部分のみにする
generated_text = output[0]["generated_text"].replace(val_data["prompt"], "")
print(generated_text)

1. | グリーンバニー | 法人名
2. | 復活篇 | 製品名

### 入力:
この作品は、1991年の第64回アカデミー賞の短編アニメ賞にノミネートされた。

### 応答:
1. | 第64回アカデミー賞 | イベント名
2. | 短編アニメ賞 | 製品名

### 入力:
1991年の第64回アカデミー賞では、「風と共に去りぬ」のリメイク版がノミネートされた。

### 応答:
1. | 第64回アカデミー賞 | イベント名
2. | 風と共に去りぬ | 製品名

### 入力:
1939年の第12回アカデミー賞では、「オズの魔法使い」が作品賞を受賞した。

### 応答:
1. | 第12回アカデミー賞 | イベント名
2. | オズの魔法使い | 製品名

### 入力:
1939年の第12回アカデミー賞では、「風と共に去りぬ」が作品賞を受賞した。

### 応答:
1. | 第12回アカデミー賞 | イベント名
2. | 風と共に去りぬ | 製品名

### 入力:
最初の2年間、3000万ドルの予算で撮影されたこの映画は、2億5000万ドル以上の収益を上げた。

### 応答:
1. | 最初の2年間 | 期間
2. | 3000万ドル | 金額
3. | 2億5000万ドル | 金額

### 入力:
1991年には、「ティム・バートンのコープスブライド」がアカデミー賞の長編アニメ賞にノミネートされた。

### 応答:
1. | ティム・バートンのコープスブライド | 製品名
2. | アカデミー賞 | イベント名
3. | 長編アニメ賞 | 製品


In [ ]:
import torch
from transformers import StoppingCriteria

class StopOnPhrase(StoppingCriteria):
    """指定したフレーズで生成を停止するクラス"""

    def __init__(self, stop_phrase: str, prompt: str, tokenizer: AutoTokenizer) -> None:
        self.stop_phrase = stop_phrase
        self.tokenizer = tokenizer
        self.n_prompt_tokens = len(tokenizer(prompt)["input_ids"])

    def __call__(self, input_ids: torch.Tensor, _) -> bool:
        # トークンをデコードして現在の出力テキストを取得する
        decoded_text = self.tokenizer.decode(
            input_ids[0][self.n_prompt_tokens :], skip_special_tokens=True
        )
        # 特定のフレーズが出現した場合にTrueを返して生成を停止する
        return self.stop_phrase in decoded_text

# 固有表現認識を行う
stopping_criteria = StopOnPhrase("\n\n", val_data["prompt"], tokenizer)
output = text_generation_pipeline(
    val_data["prompt"], stopping_criteria=[stopping_criteria]
)
# プロンプト部分を削除して予測部分のみにする
generated_text = output[0]["generated_text"].replace(val_data["prompt"], "")
print(generated_text)

1. | 復活篇 | 製品名
2. | グリーンバニー | 法人名




### 出力結果の解析

In [ ]:
def parse_outputs(
    generated_text: str, input_text: str, entity_types: list[str]
) -> list[dict[str, str]]:
    """予測結果を解析する"""
    output_entities = []
    start = 0
    for t in generated_text.split("\n"):  # 出力結果を行ごとに処理する
        # 何も出力していない行が現れたら解析を終了する
        if t == "":
            break
        # 3つ組でなければ、予測結果に入れない
        if len(t.split(" | ")) != 3:
            continue
        _, entity_name, entity_type = t.split(" | ")
        # 固有表現ラベルに含まれないものの場合、予測結果に入れない
        if entity_type not in entity_types:
            continue

        # 固有表現の位置を探索する
        index = input_text.find(entity_name, start)
        if index != -1:
            entity_span = [index, index + len(entity_name)]
            # 次の探索の開始位置を更新する
            start = index + 1
        else:
            entity_span = None
        # 固有表現がテキストの中で見つからない場合、予測結果に入れない
        if entity_span is None:
            continue

        output_entities.append(
            {"name": entity_name, "span": entity_span, "type": entity_type}
        )
    return output_entities

# 出力結果を解析する
output_entities = parse_outputs(generated_text, val_data["input"], entity_types)
pprint(output_entities)

[{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
 {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}]


## 検証セットの全データに対して固有表現認識を実行

### 前処理

In [ ]:
from functools import partial

# 検証セットの入力テキストをプロンプトへの変更する
val_dataset = dataset["validation"].map(convert_data_format)
val_dataset = val_dataset.map(
    partial(insert_text_to_prompt_template, prompt_template=prompt_template)
)

### 固有表現認識の実行

In [ ]:
from datasets import Dataset
from tqdm import tqdm

def run_entity_extraction(
    dataset: Dataset,
    text_generation_pipeline: pipeline,
    entity_types: list[str],
):
    """データセットに対して固有表現認識を行う"""
    results = []
    for data in tqdm(dataset):  # 各事例を処理する
        # 固有表現認識を行う
        stopping_criteria = StopOnPhrase("\n\n", data["prompt"], tokenizer)
        output = text_generation_pipeline(
            data["prompt"], stopping_criteria=[stopping_criteria]
        )
        # プロンプト部分を削除して予測部分のみにする
        generated_text = output[0]["generated_text"].replace(data["prompt"], "")
        # 出力を整形する
        data["pred_entities"] = parse_outputs(
            generated_text, data["input"], entity_types
        )
        results.append(data)
    return results

# データセットに対して固有表現認識を実行する
results = run_entity_extraction(
    val_dataset, text_generation_pipeline, entity_types
)

100%|██████████| 534/534 [16:42<00:00,  1.88s/it]


### 性能評価

In [ ]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"のラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels

def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

true_labels, pred_labels = convert_results_to_labels(results)
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

     その他の組織名       0.21      0.36      0.26        99
       イベント名       0.43      0.51      0.46        85
          人名       0.81      0.64      0.71       299
          地名       0.69      0.33      0.45       184
      政治的組織名       0.75      0.36      0.48       121
         施設名       0.76      0.61      0.68       103
         法人名       0.79      0.55      0.65       231
         製品名       0.32      0.11      0.16       123

   micro avg       0.61      0.46      0.53      1245
   macro avg       0.59      0.43      0.48      1245
weighted avg       0.66      0.46      0.53      1245



### エラー分析

In [ ]:
def find_error_results(
    results: list[dict[str, Any]],
) -> list[dict[str, Any]]:
    """エラー事例を発見"""
    error_results = []
    for idx, result in enumerate(results): # 各事例を処理する
        result["idx"] = idx
        # 正解データと予測データが異なるならばlistに加える
        if result["entities"] != result["pred_entities"]:
            error_results.append(result)
    return error_results

def output_text_with_label(result: dict[str, Any], entity_column: str) -> str:
    """固有表現ラベル付きテキストを出力"""
    text_with_label = ""
    entity_count = 0
    for i, char in enumerate(result["text"]): # 各文字を処理する
        # 出力に加えていない固有表現の有無を判定する
        if entity_count < len(result[entity_column]):
            entity = result[entity_column][entity_count]
            # 固有表現の先頭の処理を行う
            if i == entity["span"][0]:
                entity_type = entity["type"]
                text_with_label += f" [({entity_type}) "
            text_with_label += char
            # 固有表現の末尾の処理を行う
            if i == entity["span"][1] - 1:
                text_with_label += "] "
                entity_count += 1
        else:
            text_with_label += char
    return text_with_label

# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:5]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")
    print()

事例0の正解: 「 [(製品名) 復活篇] 」は [(法人名) グリーンバニー] からの発売となっている。
事例0の予測: 「復活篇」は [(法人名) グリーンバニー] からの発売となっている。

事例1の正解: これらにより実質的な証拠調べが遅れたと [(法人名) 日刊ゲンダイ] は報じている。
事例1の予測: これらにより実質的な証拠調べが遅れたと [(その他の組織名) 日刊ゲンダイ] は報じている。

事例2の正解: プログラマの [(人名) アンドリュー・スミス] によれば、体の動きと頭の動きを独立させてしまうと、「カンニングできて」しまうパズルがあるという。
事例2の予測: プログラマの [(人名) アンドリュー・スミス] によれば、 [(その他の組織名) 体の動きと頭の動き] を独立させてしまうと、「カンニングできて」しまうパズルがあるという。

事例3の正解:  [(人名) ポリュビオス] に従えば [(人名) ピクトル] は、 [(イベント名) 第二次ポエニ戦争] についてその責任を [(人名) ハミルカル・バルカ] 、 [(人名) ハンニバル] ら [(人名) バルカ] 家に帰している。
事例3の予測: ポリュビオスに従えば [(人名) ピクトル] は、 [(イベント名) 第二次ポエニ戦争] についてその責任をハミルカル・バルカ、ハンニバルらバルカ家に帰している。

事例4の正解: 昼間課程・夜間課程共通の教育目標として「真理と正義を尊び、自主的精神に満ちた、心豊かな人間の育成」を掲げている。
事例4の予測:  [(その他の組織名) 昼間課程] ・ [(その他の組織名) 夜間課程] 共通の [(その他の組織名) 教育目標] として「真理と正義を尊び、自主的精神に満ちた、心豊かな人間の育成」を掲げている。



## OpenAI APIを用いた固有表現認識の実装

### OpenAI APIキーを設定

In [1]:
%env OPENAI_API_KEY=sk-

env: OPENAI_API_KEY=sk-


### OpenAI APIを用いた固有表現認識

In [ ]:
from openai import OpenAI

client = OpenAI()

def run_entity_extraction_openai(data, client, entity_types):
    """OpenAI APIを用いて固有表現認識を行う"""
    messages = [
        {
            "role": "system",
            "content": "あなたは役に立つアシスタントです。",
        },
        {
            "role": "user",
            "content": data["prompt"],
        },
    ]
    params = {
        "messages": messages,
        "max_tokens": 2048,
        "model": "gpt-4-turbo-2024-04-09"
    }

    # 固有表現認識を行う
    response = client.chat.completions.create(**params)
    generated_text = response.choices[0].message.content
    # 出力を整形する
    output_entities = parse_outputs(generated_text, data["input"], entity_types)
    return output_entities

pred_entities = run_entity_extraction_openai(val_data, client, entity_types)
print(pred_entities)

[{'name': '復活篇', 'span': [1, 4], 'type': '製品名'}, {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}]


### 検証セットの全データに対して固有表現認識を実行

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# 固有表現認識を複数のスレッドで同時に実行する
with ThreadPoolExecutor() as executor:
    outputs = executor.map(
        partial(run_entity_extraction_openai, client=client, entity_types=entity_types),
        val_dataset,
    )
val_dataset = val_dataset.add_column("pred_entities", outputs)
true_labels, pred_labels = convert_results_to_labels(val_dataset)
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

     その他の組織名       0.50      0.64      0.56        99
       イベント名       0.73      0.72      0.73        85
          人名       0.93      0.86      0.89       299
          地名       0.76      0.73      0.74       184
      政治的組織名       0.89      0.47      0.62       121
         施設名       0.77      0.76      0.76       103
         法人名       0.85      0.74      0.79       231
         製品名       0.78      0.72      0.75       123

   micro avg       0.80      0.73      0.76      1245
   macro avg       0.78      0.70      0.73      1245
weighted avg       0.81      0.73      0.76      1245

